In [5]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Mokymasis visą gyvenimą tapo politikos prioritetu , o valstybių narių švietimo ir mokymo sistemų reformomis siekiama didinti investicijas į žmogiškąjį kapitalą , sudaryti palankesnes sąlygas inovacijoms ir skatinti verslumo kultūrą ."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input, output_hidden_states = True)

In [9]:
print(len(output))
hidden_states = output[2]
print(hidden_states)
print(len(hidden_states))

3
(tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [ 0.3180,  0.6312, -0.5047,  ...,  0.2151,  0.0246, -0.1945],
         [-0.1757,  0.1410,  0.2358,  ...,  0.6386,  0.4809, -0.1323],
         ...,
         [-0.3308,  0.8271, -0.2800,  ..., -0.4470, -0.5362,  0.9812],
         [-0.4498,  0.3225,  0.5143,  ...,  0.1424,  0.3526,  0.3998],
         [-0.5649, -0.1463,  0.5240,  ..., -0.7546, -0.1358, -0.2930]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[[ 0.1061, -0.0247, -0.2049,  ...,  0.2179,  0.0355, -0.0201],
         [ 0.2456,  1.2804, -0.8012,  ...,  0.5985, -0.0414, -0.2635],
         [-0.7609, -0.4656,  0.1074,  ...,  0.6016,  0.6009,  0.1062],
         ...,
         [ 0.2051,  1.1009, -0.8263,  ..., -0.7857, -0.7319,  0.8676],
         [-0.4237, -0.2488,  0.2439,  ...,  0.0347, -0.0556,  0.0653],
         [-0.2385, -0.5702,  0.1014,  ..., -0.1559,  0.0966, -0.2295]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[[ 0.0083, -0.207

In [12]:
embedding_output = hidden_states[0]
print(embedding_output)
print(len(embedding_output))

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [ 0.3180,  0.6312, -0.5047,  ...,  0.2151,  0.0246, -0.1945],
         [-0.1757,  0.1410,  0.2358,  ...,  0.6386,  0.4809, -0.1323],
         ...,
         [-0.3308,  0.8271, -0.2800,  ..., -0.4470, -0.5362,  0.9812],
         [-0.4498,  0.3225,  0.5143,  ...,  0.1424,  0.3526,  0.3998],
         [-0.5649, -0.1463,  0.5240,  ..., -0.7546, -0.1358, -0.2930]]],
       grad_fn=<NativeLayerNormBackward0>)
1


In [2]:
def read_examples(input_file):
  """Read a list of `InputExample`s from an input file."""
  examples = []
  unique_id = 0
  with open(input_file, "r", encoding='utf-8') as reader:
    while True:
      line = reader.readline()
      if not line:
          break
      text = line.strip().split('\t')[-1]
      examples.append(text)
      unique_id += 1
  return examples

In [7]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained("bert-base-multilingual-uncased")
text = read_examples("ObjCase_test.txt")
encoded_input = tokenizer(sentence, return_tensors='pt', padding = True)
output = model(**encoded_input, output_hidden_states = True)

print(output)

> c:\users\adoma\anaconda3\lib\site-packages\torch\nn\functional.py(2266)embedding()
   2264     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   2265 
-> 2266 
   2267 def embedding_bag(
   2268     input: Tensor,



ipdb>  continue


> c:\users\adoma\anaconda3\lib\site-packages\torch\nn\functional.py(2266)embedding()
   2264     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   2265 
-> 2266 
   2267 def embedding_bag(
   2268     input: Tensor,



ipdb>  continue


> c:\users\adoma\anaconda3\lib\site-packages\torch\nn\functional.py(2266)embedding()
   2264     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   2265 
-> 2266 
   2267 def embedding_bag(
   2268     input: Tensor,



ipdb>  continue


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0780, -0.0454, -0.0454,  ..., -0.0535, -0.0218, -0.0535],
         [ 0.1749,  0.6618, -0.1256,  ..., -0.0904,  0.1683, -0.2197],
         [-0.9231,  0.0877,  0.3768,  ..., -0.3216, -0.2217, -0.0118],
         ...,
         [-0.0898,  0.3302, -0.0684,  ..., -0.1578, -0.1485, -0.1357],
         [-0.4545,  0.8917, -0.2299,  ...,  0.7790, -0.1886, -1.1163],
         [-0.4443,  0.9073, -0.2187,  ...,  0.8316, -0.1154, -1.1627]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-6.3372e-02,  1.4227e-01,  5.2355e-02,  8.2032e-02,  1.1336e-01,
          2.4476e-01, -9.0115e-04,  8.7838e-02, -8.4573e-02,  8.2633e-02,
         -5.7566e-02, -1.6131e-02,  7.5042e-02, -8.4975e-02,  7.0461e-03,
          8.1162e-03, -2.7818e-02, -6.4721e-02, -2.6112e-03,  2.3068e-02,
         -3.0404e-02,  2.5914e-02, -7.6890e-02,  1.5580e-01,  1.0996e-01,
         -3.9749e-02,  1.1072e-01, -6.2109e-03, -1.6938e-01,  1.832

In [12]:
hidden_states = output[2]

In [15]:
embedding_output = hidden_states[0]
print(embedding_output)
embedding_output.shape

tensor([[[ 0.0852, -0.1362, -0.5025,  ..., -0.0093,  0.0121, -0.0160],
         [-0.7495,  0.4262, -0.3323,  ...,  0.1098,  0.6901,  0.2638],
         [ 0.7105, -0.4261,  0.1650,  ..., -0.4462, -0.3033, -0.6886],
         ...,
         [ 0.4359,  0.2011,  0.4763,  ...,  0.6097, -1.6940,  1.3611],
         [ 0.1792, -0.0962, -0.3525,  ...,  0.3006,  0.1500,  0.4358],
         [ 0.4632, -0.1756,  1.0297,  ...,  0.1870, -0.2757,  0.0040]]],
       grad_fn=<NativeLayerNormBackward0>)


torch.Size([1, 71, 768])

In [16]:
attention_states = hidden_states[1:]
print(attention_states)

(tensor([[[ 0.0094,  0.0335,  0.1026,  ...,  0.0421, -0.0373,  0.0188],
         [ 0.5988,  0.3806, -1.8592,  ...,  0.7229,  0.7661,  0.0906],
         [ 1.4152, -0.2071, -1.0702,  ..., -0.0607, -0.6562, -0.6289],
         ...,
         [ 0.8508,  0.3830,  0.0023,  ...,  0.9051, -1.6294,  1.6058],
         [ 0.1439,  0.4666,  0.5801,  ..., -0.0111, -0.0595,  0.6275],
         [ 0.6441, -0.1495,  0.7339,  ...,  0.1540, -0.3807, -0.0133]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[[-2.5855e-02,  1.4637e-01,  1.6707e-01,  ..., -4.3336e-02,
          -2.6447e-02, -1.4155e-02],
         [ 5.8624e-01,  4.0281e-02, -1.8212e+00,  ...,  7.4987e-01,
           3.7757e-01,  6.8543e-03],
         [ 2.5469e+00,  4.4087e-01, -8.9541e-01,  ...,  2.0973e-01,
          -8.4274e-01,  2.2605e-01],
         ...,
         [ 8.0662e-01,  2.1044e-01,  2.1208e-01,  ...,  3.9372e-01,
          -1.1043e+00,  1.0303e+00],
         [ 4.9275e-01,  5.0076e-01,  4.8073e-01,  ..., -3.3958e-01,
          -

In [17]:
print(attention_states[0])

tensor([[[ 0.0094,  0.0335,  0.1026,  ...,  0.0421, -0.0373,  0.0188],
         [ 0.5988,  0.3806, -1.8592,  ...,  0.7229,  0.7661,  0.0906],
         [ 1.4152, -0.2071, -1.0702,  ..., -0.0607, -0.6562, -0.6289],
         ...,
         [ 0.8508,  0.3830,  0.0023,  ...,  0.9051, -1.6294,  1.6058],
         [ 0.1439,  0.4666,  0.5801,  ..., -0.0111, -0.0595,  0.6275],
         [ 0.6441, -0.1495,  0.7339,  ...,  0.1540, -0.3807, -0.0133]]],
       grad_fn=<NativeLayerNormBackward0>)


In [18]:
attention_states[0].shape

torch.Size([1, 71, 768])

In [20]:
attention_states[0][0].shape

torch.Size([71, 768])

In [21]:
attention_states[0][0][0]

tensor([ 9.4085e-03,  3.3546e-02,  1.0265e-01,  1.2798e-01, -1.9224e-02,
         1.0460e-01,  1.1069e-01,  3.6181e-01, -7.9835e-02, -4.3085e-03,
         6.0830e-02,  3.3146e-02, -2.1032e-01, -3.7836e-02, -2.2753e-03,
         1.1189e-01,  1.1779e-01,  2.8227e-02, -2.8524e-02, -1.7341e-01,
        -1.7848e-01, -3.8752e-02,  1.4356e-01, -2.1748e-02,  1.4220e-01,
         1.7719e-01,  1.3729e-02,  1.5290e-02,  1.0853e-01, -3.6514e-02,
         3.0824e-01, -1.3306e-01,  1.1206e-01, -2.4077e-01, -2.5155e-02,
        -3.5963e-02,  1.0429e-01,  8.2572e-02, -1.0377e-01, -8.2127e-03,
         3.5695e-02,  2.0518e-02,  1.5452e-01,  1.1102e-01, -1.5344e-02,
         2.3688e-02,  1.4643e-01,  4.7333e-02, -2.5476e-03, -1.1751e-01,
        -1.6738e-01, -6.3824e-02,  7.1502e-02, -1.3477e-02, -6.3210e-02,
        -2.0636e-02,  8.5308e-02,  4.2832e-02,  3.0593e-02, -1.7998e-02,
         1.5114e-01,  7.5114e-02,  1.1214e-01, -5.4157e-02, -4.4790e-02,
         3.2222e-02,  5.7397e-02, -1.9898e-01,  1.1

In [4]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained("bert-base-multilingual-uncased")
text = read_examples("ObjCase_test.txt")
encoded_input = tokenizer(text, return_tensors='pt', padding = True)
output = model(**encoded_input, output_hidden_states = True)

print(output[0])

tensor([[[-0.0780, -0.0454, -0.0454,  ..., -0.0535, -0.0218, -0.0535],
         [ 0.1749,  0.6618, -0.1256,  ..., -0.0904,  0.1683, -0.2197],
         [-0.9231,  0.0877,  0.3768,  ..., -0.3216, -0.2217, -0.0118],
         ...,
         [-0.0898,  0.3302, -0.0684,  ..., -0.1578, -0.1485, -0.1357],
         [-0.4545,  0.8917, -0.2299,  ...,  0.7790, -0.1886, -1.1163],
         [-0.4443,  0.9073, -0.2187,  ...,  0.8316, -0.1154, -1.1627]]],
       grad_fn=<NativeLayerNormBackward0>)


In [5]:
output[0].shape

torch.Size([1, 71, 768])

In [10]:
output[2][0].shape

torch.Size([1, 71, 768])

In [8]:
import collections
import argparse
from tqdm import tqdm
import json
import torch

from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
text = read_examples("ObjCase.txt")
with open("ObjCase_multi.json", "w", encoding='utf-8') as writer:
    for sentence in text:
        encoded_input = tokenizer(sentence, return_tensors='pt', padding = True)
        output = model(**encoded_input, output_hidden_states = True)
        hidden_states = output[2][1:]
        output_json = collections.OrderedDict()
        output_json["linex_index"] = text.index(sentence)
        all_out_features = []
        all_layers = []
        for layer_index in range(len(hidden_states)):
            layer_output = hidden_states[int(layer_index)].detach().cpu().numpy()
            layer_output = layer_output[0, 0, :]
            layers = collections.OrderedDict()
            layers["index"] = layer_index
            layers["values"] = [
                round(x.item(), 6) for x in layer_output
            ]
            all_layers.append(layers)
        out_features = collections.OrderedDict()
        out_features["token"] = "[CLS]"
        out_features["layers"] = all_layers
        all_out_features.append(out_features)
        output_json["features"] = all_out_features
        writer.write(json.dumps(output_json) + "\n")